<a href="https://colab.research.google.com/github/jkchandalia/nlp/blob/main/More_SOTA_Deep_Learning_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Generative AI**

**Import** Libraries 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#### https://huggingface.co/docs/transformers/index

## GPT
##### Transformer Decoder


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)


In [ ]:
# encode context the generation is conditioned on
input_ids = tokenizer.encode('I enjoy walking with my cute dog', return_tensors='tf')

# generate text until the output length (which includes the context length) reaches 50

sample_output = model.generate(
    input_ids,
    do_sample=True, 
    max_length=50, 
    top_k=50, 
    top_p=0.95, 
)
print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))


## ChatGPT
#### https://chat.openai.com/chat

## Diffusion

#### https://huggingface.co/docs/diffusers/index

#### https://labs.openai.com/

## Bias in LLMs

https://huggingface.co/bert-base-uncased?text=The+nurse+went+for+a+walk+because+%5BMASK%5D+wanted+some+exercise

https://huggingface.co/bert-base-uncased?text=Karen+went+out+to+%5BMASK%5D+the+bank